In [33]:
import os

In [34]:
%pwd

'C:\\Users\\sar\\OneDrive\\Desktop\\DataScienceProjectE2E'

In [35]:
# os.chdir("../")
# %pwd

In [36]:
import os
import sys
os.chdir(r"C:\Users\sar\OneDrive\Desktop\DataScienceProjectE2E")
sys.path.append(os.getcwd())
%pwd


'C:\\Users\\sar\\OneDrive\\Desktop\\DataScienceProjectE2E'

In [37]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [38]:
from src.datascience.constant import (
    CONFIG_FILE_PATH,
    PARAMS_FILE_PATH,
    SCHEMA_FILE_PATH
)

from src.datascience.utils.common import read_yaml,create_directories

In [39]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

        
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config
            
            
        
        

In [40]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile


In [41]:
### components - Data - ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
        
        
     ## Downloading the zip file   
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
            
        else:
            logger.info(f"file already exist")
                    
    def extract_zip_file(self):
        """zip_file_path: str
           extract the zip file into the data directory
           function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [42]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e    

[2026-01-25 14:05:07,705: INFO: common: YAML file: config\config.yamlloaded successfully from: config\config.yaml]
[2026-01-25 14:05:07,710: INFO: common: YAML file: params.yamlloaded successfully from: params.yaml]
[2026-01-25 14:05:07,714: INFO: common: YAML file: schema.yamlloaded successfully from: schema.yaml]
[2026-01-25 14:05:07,720: INFO: common: created directory at: artifacts]
[2026-01-25 14:05:07,723: INFO: common: created directory at: artifacts/data_ingestion]


[2026-01-25 14:05:09,223: INFO: 31388440: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0D8C:15D31C:39CE16:92BA32:6975D5BC
Accept-Ranges: bytes
Date: Sun, 25 Jan 2026 08:35:08 GMT
Via: 1.1 varnish
X-Served-By: cache-bom-vanm7210031-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1769330109.657965,VS0,VE244
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 3101f8178ca29d194a0b955eab960aeeba497c7c
Expires: Sun, 25 Jan 2026 08:40:08 GMT
Source-Age: 0

]
